In [126]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools
import numpy as np

In [127]:
!pip install MechanicalSoup
import mechanicalsoup

In [155]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://godvillegame.com/")
browser.select_form('form[action="/login/login"]')
browser["username"] = "Go24yWnHZ77wL2"
browser["password"] = "go24yWnHZ77wL2"
response = browser.submit_selected()
browser.follow_link("news")

<Response [200]>

In [156]:
soup = browser.get_current_page()
#link = urlopen('https://godvillegame.com/news').read()    
#soup = BeautifulSoup(html_doc, 'html.parser')

#print(soup.prettify())

In [157]:
across_comingup = 0
down_comingup = 0
word_incoming = 0
word_index_across = []
word_index_down = []
for string in soup.find("div", attrs={"class": "cross_q"}).stripped_strings:
  line = repr(string)
  line = line.replace('\\t', '')
  line = line.replace('\\n', ' ')
  line = line.replace('&nbsp', ' ')
  line = line.replace('.', '')
  line = line.replace('\'', '')
  if across_comingup:
    splitted = line.split()
    iter_number = len(splitted)
    for i in range(iter_number):
      if word_incoming:
        word_incoming = 0
      if splitted[i].isdigit():
        word_index_across.append(int(splitted[i]))
        word_incoming = 1  
  if down_comingup:
    splitted = line.split()
    iter_number = len(splitted)
    for i in range(iter_number):
      if word_incoming:
        word_incoming = 0
      if splitted[i].isdigit():
        word_index_down.append(int(splitted[i]))
        word_incoming = 1  
  if line == "Across:":
    across_comingup = 1
  elif line == "Down:":
    across_comingup = 0
    down_comingup = 1

In [158]:
across_words = [[] for i in range(len(word_index_across))]
down_words = [[] for i in range(len(word_index_down))]
list_of_unfilled_letter_positions = [[]]
counter = 0
for unknown_letters in soup.find_all("input", attrs={"autocomplete":"off"}):
  line = unknown_letters['aria-label']
  line = line.replace(',', '')
  line = line.replace('.', ' .')
  splitted = line.split()
  #print(splitted)
  if (splitted[1] == 'down'):
    down_words[word_index_down.index(int(splitted[0]))].append([])
    list_of_unfilled_letter_positions[counter].append(1)
    list_of_unfilled_letter_positions[counter].append(word_index_down.index(int(splitted[0])))
    list_of_unfilled_letter_positions[counter].append(int(splitted[splitted.index('letter') + 1]))
    list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
  if (splitted[1] == 'across'):
    across_words[word_index_across.index(int(splitted[0]))].append([])
    list_of_unfilled_letter_positions[counter].append(0)
    list_of_unfilled_letter_positions[counter].append(word_index_across.index(int(splitted[0])))
    list_of_unfilled_letter_positions[counter].append(int(splitted[splitted.index('letter') + 1]))
    list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
  if '.' in splitted :
    shifted_index = splitted.index('.') + 1
    counter += 1
    list_of_unfilled_letter_positions.append([])
    if (splitted[shifted_index + 1] == 'down'):
      down_words[word_index_down.index(int(splitted[shifted_index + 0]))].append([])
      list_of_unfilled_letter_positions[counter].append(1)
      list_of_unfilled_letter_positions[counter].append(word_index_down.index(int(splitted[shifted_index + 0])))
      list_of_unfilled_letter_positions[counter].append(int(splitted[splitted[shifted_index:].index('letter') + 1 + shifted_index]))
      list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
    if (splitted[shifted_index + 1] == 'across'):
      across_words[word_index_across.index(int(splitted[shifted_index + 0]))].append([])
      list_of_unfilled_letter_positions[counter].append(0)
      list_of_unfilled_letter_positions[counter].append(word_index_across.index(int(splitted[shifted_index + 0])))
      list_of_unfilled_letter_positions[counter].append(int(splitted[splitted[shifted_index:].index('letter') + 1 + shifted_index]))
      list_of_unfilled_letter_positions[counter].append(unknown_letters['name'])
  counter += 1
  list_of_unfilled_letter_positions.append([])
list_of_unfilled_letter_positions.pop()

[]

In [159]:
for unknown_letters in soup.find_all("div", attrs={"class": "open"}):
  line = unknown_letters['aria-label']
  line = line.replace(',', '')
  line = line.replace('-', '')
  splitted = line.split()
  #print(splitted)
  if (splitted[1] == 'down'):
    down_words[word_index_down.index(int(splitted[0]))].append([])
  if (splitted[1] == 'across'):
    across_words[word_index_across.index(int(splitted[0]))].append([])

In [160]:
for unknown_letters in soup.find_all("div", attrs={"class": "open"}):
  line = unknown_letters['aria-label']
  line = line.replace(',', '')
  line = line.replace('-', '')
  splitted = line.split()
  #print(splitted)
  if splitted[-1].isdigit(): 
    if (splitted[1] == 'down'):
      down_words[word_index_down.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = ' '
    elif (splitted[1] == 'across'):
      across_words[word_index_across.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = ' '
  else:
    if (splitted[1] == 'down'):
      down_words[word_index_down.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = splitted[-1]
    elif (splitted[1] == 'across'):
      across_words[word_index_across.index(int(splitted[0]))][int(splitted[splitted.index("letter") + 1]) - 1] = splitted[-1]

In [161]:
link_solution = urlopen('https://wiki.godvillegame.com/Omnibus_List').read()    
soup_solution = BeautifulSoup(link_solution, 'html.parser')

#print(soup_solution.prettify())

In [162]:
across_solution = [[] for i in range(len(word_index_across))]
down_solution = [[] for i in range(len(word_index_down))]
for i in range(len(across_words)):
  l_test = across_words[i]
  for string in soup_solution.find_all("li"):
    line = list(string.text.upper())
    count = np.where([ x == y for x, y in zip(line, l_test) ])[0].size
    if count == len(np.where([x != [] for x in l_test])[0]):
      print(line)
      across_solution[i] = line
for i in range(len(down_words)):
  l_test = down_words[i]
  for string in soup_solution.find_all("li"):
    line = list(string.text.upper())
    count = np.where([ x == y for x, y in zip(line, l_test) ])[0].size
    if count == len(np.where([x != [] for x in l_test])[0]) and len(line) == len(l_test):
      print(line)
      down_solution[i] = line

['N', 'O', 'N', 'D', 'E', 'S', 'C', 'R', 'I', 'P', 'T', ' ', 'V', 'I', 'L', 'L', 'A', 'I', 'N']
['M', 'I', 'N', 'T', '-', 'C', 'O', 'N', 'D', 'I', 'T', 'I', 'O', 'N', ' ', 'M', 'I', 'N', 'T']
['M', 'A', 'L', 'N', 'U', 'T', 'R', 'I', 'T', 'I', 'O', 'N', ' ', 'S', 'U', 'P', 'P', 'L', 'E', 'M', 'E', 'N', 'T']
['T', 'E', 'M', 'P', 'L', 'E', ' ', 'S', 'A', 'F', 'E', 'T', 'Y', ' ', 'I', 'N', 'S', 'P', 'E', 'C', 'T', 'O', 'R']
['I', 'N', 'D', 'I', 'F', 'F', 'E', 'R', 'E', 'N', 'C', 'E', ' ', 'E', 'N', 'G', 'I', 'N', 'E']
['S', 'U', 'P', 'E', 'R', 'S', 'O', 'N', 'I', 'C', ' ', 'H', 'E', 'D', 'G', 'E', 'H', 'O', 'G']
['P', 'S', 'I', 'O', 'N', 'I', 'C', ' ', 'T', 'O', 'N', 'I', 'C']
['R', 'O', 'O', 'T', 'B', 'E', 'A', 'R']
['J', 'A', 'N', 'I', 'T', 'A', 'U', 'R']
['M', 'I', 'N', 'I', 'T', 'A', 'U', 'R']


In [167]:
print(browser.get_url())
browser.select_form()

https://godvillegame.com/news?c%5B2%5D%5B0%5D=N&c%5B3%5D%5B0%5D=O&c%5B4%5D%5B0%5D=N&c%5B5%5D%5B0%5D=D&c%5B6%5D%5B0%5D=E&c%5B7%5D%5B0%5D=S&c%5B8%5D%5B0%5D=C&c%5B9%5D%5B0%5D=R&c%5B10%5D%5B0%5D=I&c%5B11%5D%5B0%5D=P&c%5B12%5D%5B0%5D=T&c%5B13%5D%5B0%5D=+&c%5B14%5D%5B0%5D=V&c%5B15%5D%5B0%5D=I&c%5B16%5D%5B0%5D=L&c%5B17%5D%5B0%5D=L&c%5B18%5D%5B0%5D=A&c%5B19%5D%5B0%5D=I&c%5B20%5D%5B0%5D=N&c%5B11%5D%5B1%5D=S&c%5B22%5D%5B1%5D=R&c%5B2%5D%5B2%5D=M&c%5B3%5D%5B2%5D=I&c%5B4%5D%5B2%5D=N&c%5B5%5D%5B2%5D=T&c%5B6%5D%5B2%5D=-&c%5B7%5D%5B2%5D=C&c%5B8%5D%5B2%5D=O&c%5B9%5D%5B2%5D=N&c%5B10%5D%5B2%5D=D&c%5B11%5D%5B2%5D=I&c%5B12%5D%5B2%5D=T&c%5B13%5D%5B2%5D=I&c%5B14%5D%5B2%5D=O&c%5B15%5D%5B2%5D=N&c%5B16%5D%5B2%5D=+&c%5B17%5D%5B2%5D=M&c%5B18%5D%5B2%5D=I&c%5B19%5D%5B2%5D=N&c%5B20%5D%5B2%5D=T&c%5B22%5D%5B2%5D=O&c%5B11%5D%5B3%5D=O&c%5B22%5D%5B3%5D=O&c%5B0%5D%5B4%5D=M&c%5B1%5D%5B4%5D=A&c%5B2%5D%5B4%5D=L&c%5B3%5D%5B4%5D=N&c%5B4%5D%5B4%5D=U&c%5B5%5D%5B4%5D=T&c%5B6%5D%5B4%5D=R&c%5B7%5D%5B4%5D=I&c%5B8%5D%5B4%5D=T&c%5B9%5

In [164]:
for positions in list_of_unfilled_letter_positions:
  if positions[0] == 0:
    browser[positions[3]] = across_solution[positions[1]][positions[2]-1]
  if positions[0] == 1:
    browser[positions[3]] = down_solution[positions[1]][positions[2]-1]

In [165]:
response = browser.submit_selected()

In [ ]:
print(response.text)

In [ ]:
git 